In [1]:
import os
import numpy as np

In [2]:
jax_data_dir = "/global/scratch/users/aniketh/promoter_modelling/jax_data/"
scripts_dir = "/global/home/users/aniketh/promoter/"

oracle_test_data_path = os.path.join(jax_data_dir, "finetune_data.pkl")
assert os.path.exists(oracle_test_data_path)

saved_models_dir = os.path.join(jax_data_dir, "saved_DEN_models_v4_TITAN")
if not os.path.exists(saved_models_dir):
    os.mkdir(saved_models_dir)
assert os.path.exists(saved_models_dir)

wandb_dir = os.path.join(jax_data_dir, "wandb_v4_TITAN")
if not os.path.exists(wandb_dir):
    os.mkdir(wandb_dir)
assert os.path.exists(wandb_dir)

bash_scripts_dir = "/global/home/users/aniketh/promoter/promoter/run_DENs_v4_TITAN_bash_scripts"
if not os.path.exists(bash_scripts_dir):
    os.mkdir(bash_scripts_dir)
    
total_count = 0
for model in reversed(sorted(os.listdir(os.path.join(jax_data_dir, "trained_predictors_after_normalization")))):
    for diff_exp_cell_ind, cell in enumerate(["THP1", "Jurkat", "K562"]):
        total_count += 1

num_splits = 2
runs_per_job = int(np.ceil(total_count / num_splits))
job_cnt = 0
cur_cnt = 0

g = open(os.path.join(bash_scripts_dir, f"train_all_DENs_j{job_cnt}.sh"), "w+")
for model in reversed(sorted(os.listdir(os.path.join(jax_data_dir, "trained_predictors_after_normalization")))):
    for diff_exp_cell_ind, cell in enumerate(["THP1", "Jurkat", "K562"]):
        experiment_id = f"DEN_v4_optimize_{cell}_using_{model}_TITAN"
        pretrained_predictor_path = os.path.join(jax_data_dir, "trained_predictors_after_normalization", model)
        
        if cur_cnt == runs_per_job:
            job_cnt += 1
            cur_cnt = 0
            g.close()
            g = open(os.path.join(bash_scripts_dir, f"train_all_DENs_j{job_cnt}.sh"), "w+")
            print("Created new job split")
        
#         cmd = f"sbatch --requeue --export=scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},diff_exp_cell_ind={diff_exp_cell_ind},oracle_test_data_path={oracle_test_data_path},saved_models_dir={saved_models_dir},wandb_dir={wandb_dir},experiment_id={experiment_id} run_DENs_v4_TITAN.sh"
#         os.system(cmd)
#         print(cmd)
#         print()
        
        f = open(os.path.join(bash_scripts_dir, experiment_id + ".sh"), "w+")
        f.write("#!/bin/bash\n")
        all_exports = f"scripts_dir={scripts_dir},pretrained_predictor_path={pretrained_predictor_path},diff_exp_cell_ind={diff_exp_cell_ind},oracle_test_data_path={oracle_test_data_path},saved_models_dir={saved_models_dir},wandb_dir={wandb_dir},experiment_id={experiment_id}"
        for ex in all_exports.split(","):
            var, val = ex.split("=")
            print(f"export {var}={val}")
            f.write(f"export {var}={val}\n")
        print("bash {}".format(os.path.join(scripts_dir, "promoter/run_DENs_v4_TITAN.sh")))
        f.write("bash {}\n".format(os.path.join(scripts_dir, "promoter/run_DENs_v4_TITAN.sh")))
        f.close()
        
        g.write("bash {}".format(os.path.join(bash_scripts_dir, experiment_id + ".sh\n")))        
        print()
        
        cur_cnt += 1
g.close()

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_after_normalization/coms_0.3_thp1_1.0.pkl
export diff_exp_cell_ind=0
export oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/finetune_data.pkl
export saved_models_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/saved_DEN_models_v4_TITAN
export wandb_dir=/global/scratch/users/aniketh/promoter_modelling/jax_data/wandb_v4_TITAN
export experiment_id=DEN_v4_optimize_THP1_using_coms_0.3_thp1_1.0.pkl_TITAN
bash /global/home/users/aniketh/promoter/promoter/run_DENs_v4_TITAN.sh

export scripts_dir=/global/home/users/aniketh/promoter/
export pretrained_predictor_path=/global/scratch/users/aniketh/promoter_modelling/jax_data/trained_predictors_after_normalization/coms_0.3_thp1_1.0.pkl
export diff_exp_cell_ind=1
export oracle_test_data_path=/global/scratch/users/aniketh/promoter_modelling/j